### 전이 학습(Transfer Learning)

전이학습이란 기존의 잘 알려진 데이터 혹은 사전학습 된 모델을 업무 효율 증대나 도메인 확장을 위해 사용하는 학습을 의미한다. 따라서 전이학습은 인공지능 분야에서 매우 중요한 연구 중 하나이며 다양한 방법론들이 존재한다.

이미 학습된 모델의 지식을 새로운 문제에 활용한다. 이미 어떤 일을 잘하는 똑똑한 모델한테, 새로운 일을 좀 더 가르쳐 새로운 문제도 잘 풀게 학습시키는 맥락이다.

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.optim as optim

from tqdm import trange 

#### 1. GPU 연산 확인

빠른 병렬 처리를 위하여 CPU보다는 GPU연산이 사용된다. CUDA를 사용할 수 있는 NVIDIA 그래픽이 있다면 CUDA 버전과 torch 버전을 맞춰 본인의 컴퓨터를 세팅하면 된다.

In [3]:
# 현재 가능한 장치 확인
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print(f'{device} is available.')

mps is available.


#### 2. CIFAR10 데이터 불러오기

In [4]:
# Class
# 'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'

# 전처리 정의(transform)
# trainset
# 여러 이미지 전처리 과정을 순서대로 적용
train_transform = transforms.Compose(
    [transforms.RandomCrop(32, padding=4), # 32x32 크기로 이미지를 랜덤하게 자름, 잘리기 전에 각 가장자리에 4픽셀 씩 padding을 넣어줘 다양한 데이터가 생성 되게 함 (Data Augmentation)
     transforms.ToTensor(), # PIL 이미지 또는 numpy array를 Pytorch Tensor로 변환, 이미지 픽셀 값을 0~1 범위로 바꿔준다. (원래 각 픽셀은 0~255 사이의 정수값)
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # 채널(RGB)마다 평균 0.5, 표준 편차 0.5로 정규화 한다. -> 결과적으로 0~1 범위에서 -1~1 범위로 스케일이 변경된다. (대부분의 신경망은 입력값이 평균 0에 가까울 때 더 빠르고 안정적으로 학습된다.)

# testset 전용 전처리 정의
# 전처리 과정은 trainset과 동일하지만, 증강(RandomCrop) 없이 정규화만 한다.
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# CIFAR-10 학습 데이터셋 불러오기
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)

# 학습용 데이터 로더 정의
# 데이터셋을 모델에 넣기 쉽게 배치 단위로 나눠준다.
# 매 epoch마다 데이터를 섞어서 학습에 편향이 없도록 한다.
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True) 

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
# 테스트셋은 평가용이기 때문에 섞지 않는다.
testloader = torch.utils.data.DataLoader(testset, batch_size=16,shuffle=False)

100%|██████████| 170M/170M [00:19<00:00, 8.93MB/s] 


#### 3. Pretrained Model 불러오기
파이토치에서는 다양한 사전 학습 모델을 제공하고 있다.

In [ ]:
# 사전 학습된 ResNet-18 불러오기
# weights='DEFAULT'를 하면 ResNet18 IMAGENET1K_V1 구조와 사전 학습 된 파라미터를 모두 불러온다.
# weights=False를 하면 ResNet18 구조만 불러온다.
# 모델과 텐서에 .to(device)를 붙여야만 GPU 연산이 가능하다.

model = torchvision.models.resnet18(weights='DEFAULT')